В коде, прилагающемся к последней лекции про обучение с подкреплением, реализован
Environment для крестиков-ноликов, в котором можно при инициализации указывать
разные размеры доски и условия победы, а также функции для рисования, в том числе с
указанием оценки различных действий. С этим окружением все задания и связаны.
1. Реализуйте обычное (табличное) Q-обучение. Обучите стратегии крестиков и
ноликов для доски 3х3.
2. Попробуйте обучить стратегии крестиков и ноликов для доски 4х4 и/или 5х5.

In [1]:
import gym
from gym import make
import numpy as np
from collections import deque, defaultdict
import random
import os
import copy

In [2]:
N_ROWS, N_COLS, N_WIN = 3, 3, 3

class TicTacToe(gym.Env):
    def __init__(self, n_rows=N_ROWS, n_cols=N_COLS, n_win=N_WIN):
        self.n_rows = n_rows
        self.n_cols = n_cols
        self.n_win = n_win

        self.board = np.zeros((self.n_rows, self.n_cols), dtype=int)
        self.gameOver = False
        self.boardHash = None
        # ход первого игрока
        self.curTurn = 1
        self.emptySpaces = None
        
        self.reset()

    def getEmptySpaces(self):
        if self.emptySpaces is None:
            res = np.where(self.board == 0)
            self.emptySpaces = np.array([ self.int_from_action((i, j)) for i,j in zip(res[0], res[1]) ])
        return self.emptySpaces

    def makeMove(self, player, i, j):
        self.board[i, j] = player
        self.emptySpaces = None
        self.boardHash = None

    def getHash(self):
        if self.boardHash is None:
            self.boardHash = ''.join(['%s' % (x+1) for x in self.board.reshape(self.n_rows * self.n_cols)])
        return self.boardHash

    def _check_terminal(self, cur_p):
        cur_marks = np.where(self.board == cur_p)
        for i,j in zip(cur_marks[0], cur_marks[1]):
            if i <= self.n_rows - self.n_win:
                if np.all(self.board[i:i+self.n_win, j] == cur_p):
                    return True
            if j <= self.n_cols - self.n_win:
                if np.all(self.board[i,j:j+self.n_win] == cur_p):
                    return True
            if i <= self.n_rows - self.n_win and j <= self.n_cols - self.n_win:
                if np.all(np.array([ self.board[i+k,j+k] == cur_p for k in range(self.n_win) ])):
                    return True
            if i <= self.n_rows - self.n_win and j >= self.n_win-1:
                if np.all(np.array([ self.board[i+k,j-k] == cur_p for k in range(self.n_win) ])):
                    return True
        return False
    
    def isTerminal(self):
        # проверим, не закончилась ли игра
        cur_win = self._check_terminal(self.curTurn)
        if cur_win:
                self.gameOver = True
                return self.curTurn
            
        if len(self.getEmptySpaces()) == 0:
            self.gameOver = True
            return 0

        self.gameOver = False
        return None

    def getWinner(self):
        # фактически запускаем isTerminal два раза для крестиков и ноликов
        if self._check_terminal(1):
            return 1
        if self._check_terminal(-1):
            return -1
        if len(self.getEmptySpaces()) == 0:
            return 0
        return None
    
    def printBoard(self):
        for i in range(0, self.n_rows):
            print('----'*(self.n_cols)+'-')
            out = '| '
            for j in range(0, self.n_cols):
                if self.board[i, j] == 1:
                    token = 'x'
                if self.board[i, j] == -1:
                    token = 'o'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' | '
            print(out)
        print('----'*(self.n_cols)+'-')

    def getState(self):
        return (self.getHash(), self.getEmptySpaces(), self.curTurn)

    def action_from_int(self, action_int):
        return ( int(action_int / self.n_cols), int(action_int % self.n_cols))

    def int_from_action(self, action):
        return action[0] * self.n_cols + action[1]
        
    def sample_action(self):
        self.getEmptySpaces()
        return np.random.choice(self.emptySpaces)
    
    def step(self, action):
        action = self.action_from_int(action)
        if self.board[action[0], action[1]] != 0:
            return self.getState(), -10, True, {}
        self.makeMove(self.curTurn, action[0], action[1])
        reward = self.isTerminal()
        self.curTurn = -self.curTurn
        return self.getState(), 0 if reward is None else reward, reward is not None, {}

    def reset(self):
        self.board = np.zeros((self.n_rows, self.n_cols), dtype=int)
        self.boardHash = None
        self.gameOver = False
        self.emptySpaces = None
        self.curTurn = 1
        return self.getState()

In [7]:
class QLearning:
    def __init__(self, state_dim, action_dim):
        self.steps = 0
        self.qvalues = defaultdict(lambda: np.zeros(action_dim))
        
    def get_qvalue(self, state, action):
        board_hash, _, _ = state
        return self.qvalues[board_hash][action]
    
    def get_value(self, state):
        best_action = self.get_best_action(state)
        if best_action is None:
            return 0.0
        return self.get_qvalue(state, best_action)
        
    def update_qvalue(self, state, action, value):
        board_hash, _, _ = state
        self.qvalues[board_hash][action] = value
        
    def get_best_action(self, state):
        _, possible_actions, _ = state
        
        best_action = None
        best_qvalue = None
        for action in possible_actions:
            qvalue = self.get_qvalue(state, action)
            if best_action is None or qvalue > best_qvalue:
                best_action = action
                best_qvalue = qvalue
                
        return best_action
        
    def act(self, state):
        return self.get_best_action(state)

    def update(self, transition):
        state, action, next_state, reward, done = transition
        qvalue = (1 - ALPHA) * self.get_qvalue(state, action) + ALPHA * (reward + GAMMA * self.get_value(next_state))
        self.update_qvalue(state, action, qvalue)
        self.steps += 1

def evaluate_policy(agent, episodes=5, crosses=True):
    max_steps = 1000
    env = TicTacToe(n_rows=N_ROWS, n_cols=N_COLS, n_win=N_WIN)
    returns = []
    for _ in range(episodes):
        done = False
        state = env.reset()
        if not crosses:
            state, reward, done, _= env.step(agent.act(state))

        total_reward = 0.        
        steps = 0
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(agent.act(state))
            # ход соперника
            if not done:
                state, reward, done, _ = env.step(env.sample_action())
                
            total_reward += reward
            steps += 1
        returns.append(total_reward)
    return returns

# Учим крестики

In [10]:
ALPHA = 0.25
GAMMA = 0.98
TRANSITIONS = 10000
STEPS_PER_UPDATE = 4
STEPS_PER_TARGET_UPDATE = STEPS_PER_UPDATE * 1000
BATCH_SIZE = 128
LEARNING_RATE = 1e-3

SEED = 13
np.random.seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

env = TicTacToe()
ql = QLearning(state_dim=(N_ROWS, N_COLS), action_dim=N_ROWS * N_COLS)
eps = 0.1
state = env.reset()
    
for i in range(TRANSITIONS):
    if random.random() < eps:
        action = env.sample_action()
    else:
        action = ql.act(state)

    next_state, reward, done, _ = env.step(action)
    # ход ноликов
    if not done:
        next_state, reward, done, _ = env.step(env.sample_action())
    
    ql.update((state, action, next_state, reward, done))

    state = next_state if not done else env.reset()

    if (i + 1) % (TRANSITIONS//100) == 0:
        rewards = evaluate_policy(ql, 500)
        print(f"Step: {i+1}, Reward mean: {np.mean(rewards)}, Reward std: {np.std(rewards)}")

Step: 100, Reward mean: 0.624, Reward std: 0.7659138332736914
Step: 200, Reward mean: 0.704, Reward std: 0.6696148146509304
Step: 300, Reward mean: 0.698, Reward std: 0.6773448161756315
Step: 400, Reward mean: 0.686, Reward std: 0.695272608406228
Step: 500, Reward mean: 0.738, Reward std: 0.6521932229025383
Step: 600, Reward mean: 0.784, Reward std: 0.5808132229899728
Step: 700, Reward mean: 0.768, Reward std: 0.6051247805205138
Step: 800, Reward mean: 0.748, Reward std: 0.6200774145217676
Step: 900, Reward mean: 0.682, Reward std: 0.6759260314561054
Step: 1000, Reward mean: 0.788, Reward std: 0.5394960611533693
Step: 1100, Reward mean: 0.814, Reward std: 0.5093171899710435
Step: 1200, Reward mean: 0.816, Reward std: 0.4838842836877429
Step: 1300, Reward mean: 0.808, Reward std: 0.512967835248956
Step: 1400, Reward mean: 0.842, Reward std: 0.46587122684278326
Step: 1500, Reward mean: 0.822, Reward std: 0.5082479709747989
Step: 1600, Reward mean: 0.834, Reward std: 0.48419417592532027
S

# Учим нолики

In [14]:
ALPHA = 0.25
GAMMA = 0.98
TRANSITIONS = 10000
STEPS_PER_UPDATE = 4
STEPS_PER_TARGET_UPDATE = STEPS_PER_UPDATE * 1000
BATCH_SIZE = 128
LEARNING_RATE = 1e-3

SEED = 13
np.random.seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

env = TicTacToe()
ql = QLearning(state_dim=(N_ROWS, N_COLS), action_dim=N_ROWS * N_COLS)
eps = 0.1
state = env.reset()
    
state, reward, done, _ = env.step(env.sample_action())
for i in range(TRANSITIONS):
    if random.random() < eps:
        action = env.sample_action()
    else:
        action = ql.act(state)

    next_state, reward, done, _ = env.step(action)
    # ход крестиков
    if not done:
        next_state, reward, done, _ = env.step(env.sample_action())
    
    ql.update((state, action, next_state, -reward, done))

    if done:
        env.reset()
        state, reward, done, _ = env.step(env.sample_action())
    else:
        state = next_state

    if (i + 1) % (TRANSITIONS//100) == 0:
        rewards = evaluate_policy(ql, 500, crosses=False)
        print(f"Step: {i+1}, Reward mean: {np.mean(rewards)}, Reward std: {np.std(rewards)}")

Step: 100, Reward mean: 0.594, Reward std: 0.7675701922300006
Step: 200, Reward mean: 0.596, Reward std: 0.7048290572897801
Step: 300, Reward mean: 0.584, Reward std: 0.6978137287270867
Step: 400, Reward mean: 0.554, Reward std: 0.6834354395259292
Step: 500, Reward mean: 0.54, Reward std: 0.698856208386246
Step: 600, Reward mean: 0.542, Reward std: 0.6871942956689905
Step: 700, Reward mean: 0.488, Reward std: 0.725159292845372
Step: 800, Reward mean: 0.512, Reward std: 0.7333866647274139
Step: 900, Reward mean: 0.608, Reward std: 0.6498738339093212
Step: 1000, Reward mean: 0.508, Reward std: 0.7388748202503589
Step: 1100, Reward mean: 0.406, Reward std: 0.7727638707910717
Step: 1200, Reward mean: 0.484, Reward std: 0.7414472334562993
Step: 1300, Reward mean: 0.456, Reward std: 0.7483742379317985
Step: 1400, Reward mean: 0.482, Reward std: 0.7467770751703617
Step: 1500, Reward mean: 0.37, Reward std: 0.8106170489201422
Step: 1600, Reward mean: 0.268, Reward std: 0.8580069929784955
Step:

# Учим крестики, 4x4x4

In [15]:
N_ROWS, N_COLS, N_WIN = 4, 4, 4

ALPHA = 0.25
GAMMA = 0.98
TRANSITIONS = 10000
STEPS_PER_UPDATE = 4
STEPS_PER_TARGET_UPDATE = STEPS_PER_UPDATE * 1000
BATCH_SIZE = 128
LEARNING_RATE = 1e-3

SEED = 13
np.random.seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

env = TicTacToe(n_rows=N_ROWS, n_cols=N_COLS, n_win=N_WIN)
ql = QLearning(state_dim=(N_ROWS, N_COLS), action_dim=N_ROWS * N_COLS)
eps = 0.1
state = env.reset()
    
for i in range(TRANSITIONS):
    if random.random() < eps:
        action = env.sample_action()
    else:
        action = ql.act(state)

    next_state, reward, done, _ = env.step(action)
    # ход ноликов
    if not done:
        next_state, reward, done, _ = env.step(env.sample_action())
    
    ql.update((state, action, next_state, reward, done))

    state = next_state if not done else env.reset()

    if (i + 1) % (TRANSITIONS//100) == 0:
        rewards = evaluate_policy(ql, 500)
        print(f"Step: {i+1}, Reward mean: {np.mean(rewards)}, Reward std: {np.std(rewards)}")

Step: 100, Reward mean: 0.458, Reward std: 0.8533674472347771
Step: 200, Reward mean: 0.492, Reward std: 0.8354256400183082
Step: 300, Reward mean: 0.5, Reward std: 0.8449852069711044
Step: 400, Reward mean: 0.44, Reward std: 0.8452218643646174
Step: 500, Reward mean: 0.512, Reward std: 0.8329801918413187
Step: 600, Reward mean: 0.542, Reward std: 0.8026431336528084
Step: 700, Reward mean: 0.478, Reward std: 0.8375655198251657
Step: 800, Reward mean: 0.494, Reward std: 0.8282294368108393
Step: 900, Reward mean: 0.506, Reward std: 0.8258111164182763
Step: 1000, Reward mean: 0.46, Reward std: 0.8534635317340747
Step: 1100, Reward mean: 0.452, Reward std: 0.8483489847934045
Step: 1200, Reward mean: 0.536, Reward std: 0.8054216287138062
Step: 1300, Reward mean: 0.492, Reward std: 0.8401999761961434
Step: 1400, Reward mean: 0.494, Reward std: 0.8282294368108392
Step: 1500, Reward mean: 0.466, Reward std: 0.8537236086696912
Step: 1600, Reward mean: 0.554, Reward std: 0.7893567001045853
Step:

# Учим нолики 4x4x4

In [19]:
import time

N_ROWS, N_COLS, N_WIN = 4, 4, 4

ALPHA = 0.25
GAMMA = 0.98
TRANSITIONS = 100000
STEPS_PER_UPDATE = 4
STEPS_PER_TARGET_UPDATE = STEPS_PER_UPDATE * 1000
BATCH_SIZE = 128
LEARNING_RATE = 1e-3

SEED = 13
np.random.seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

env = TicTacToe(n_rows=N_ROWS, n_cols=N_COLS, n_win=N_WIN)
ql = QLearning(state_dim=(N_ROWS, N_COLS), action_dim=N_ROWS * N_COLS)
eps = 0.1
state = env.reset()
    
state, reward, done, _ = env.step(env.sample_action())
for i in range(TRANSITIONS):
    if random.random() < eps:
        action = env.sample_action()
    else:
        action = ql.act(state)

    next_state, reward, done, _ = env.step(action)
    # ход крестиков
    if not done:
        next_state, reward, done, _ = env.step(env.sample_action())
    
    ql.update((state, action, next_state, -reward, done))

    if done:
        env.reset()
        state, reward, done, _ = env.step(env.sample_action())
    else:
        state = next_state

    if (i + 1) % (TRANSITIONS//1000) == 0:
        rewards = evaluate_policy(ql, 500, crosses=False)
        print(f"Step: {i+1}, Reward mean: {np.mean(rewards)}, Reward std: {np.std(rewards)}")

Step: 100, Reward mean: 0.276, Reward std: 0.6984439848692234
Step: 200, Reward mean: 0.252, Reward std: 0.7186765614655873
Step: 300, Reward mean: 0.208, Reward std: 0.7514891882123123
Step: 400, Reward mean: 0.206, Reward std: 0.7612910087476406
Step: 500, Reward mean: 0.238, Reward std: 0.7303122619811337
Step: 600, Reward mean: 0.246, Reward std: 0.7439650529426769
Step: 700, Reward mean: 0.242, Reward std: 0.7425873685971234
Step: 800, Reward mean: 0.284, Reward std: 0.7289334674714834
Step: 900, Reward mean: 0.25, Reward std: 0.7453187237685633
Step: 1000, Reward mean: 0.246, Reward std: 0.7439650529426769
Step: 1100, Reward mean: 0.208, Reward std: 0.772486893092692
Step: 1200, Reward mean: 0.236, Reward std: 0.7512017039384294
Step: 1300, Reward mean: 0.248, Reward std: 0.7446448818060861
Step: 1400, Reward mean: 0.184, Reward std: 0.7335829878071055
Step: 1500, Reward mean: 0.21, Reward std: 0.7388504584826351
Step: 1600, Reward mean: 0.21, Reward std: 0.762823701781742
Step: 

Step: 37700, Reward mean: 0.206, Reward std: 0.7480401058766837
Step: 37800, Reward mean: 0.198, Reward std: 0.7528585524519197
Step: 37900, Reward mean: 0.226, Reward std: 0.7368337668701129
Step: 38000, Reward mean: 0.242, Reward std: 0.7559338595406347
Step: 38100, Reward mean: 0.226, Reward std: 0.7476121989373903
Step: 38200, Reward mean: 0.152, Reward std: 0.7828767463656077
Step: 38300, Reward mean: 0.196, Reward std: 0.7467154745952437
Step: 38400, Reward mean: 0.194, Reward std: 0.7323687595740277
Step: 38500, Reward mean: 0.222, Reward std: 0.713243296498467
Step: 38600, Reward mean: 0.268, Reward std: 0.7511165022817698
Step: 38700, Reward mean: 0.256, Reward std: 0.7255783899758868
Step: 38800, Reward mean: 0.15, Reward std: 0.7612489737267303
Step: 38900, Reward mean: 0.196, Reward std: 0.7547078905112892
Step: 39000, Reward mean: 0.206, Reward std: 0.7507089982143548
Step: 39100, Reward mean: 0.23, Reward std: 0.7355949972641196
Step: 39200, Reward mean: 0.232, Reward std

Step: 50900, Reward mean: 0.242, Reward std: 0.7716449960960027
Step: 51000, Reward mean: 0.18, Reward std: 0.7613146524269713
Step: 51100, Reward mean: 0.158, Reward std: 0.7463484440929719
Step: 51200, Reward mean: 0.214, Reward std: 0.7537930750544211
Step: 51300, Reward mean: 0.234, Reward std: 0.7288648708779976
Step: 51400, Reward mean: 0.192, Reward std: 0.7260413211381291
Step: 51500, Reward mean: 0.25, Reward std: 0.7179832867135557
Step: 51600, Reward mean: 0.242, Reward std: 0.7398891808913007
Step: 51700, Reward mean: 0.254, Reward std: 0.7412718799468925
Step: 51800, Reward mean: 0.224, Reward std: 0.7083953698324122
Step: 51900, Reward mean: 0.224, Reward std: 0.7468761610869636
Step: 52000, Reward mean: 0.25, Reward std: 0.7399324293474371
Step: 52100, Reward mean: 0.226, Reward std: 0.7286453183819958
Step: 52200, Reward mean: 0.14, Reward std: 0.7644605941446558
Step: 52300, Reward mean: 0.192, Reward std: 0.7450744929200033
Step: 52400, Reward mean: 0.284, Reward std:

Step: 79200, Reward mean: 0.234, Reward std: 0.7424580796247018
Step: 79300, Reward mean: 0.196, Reward std: 0.7833160281776443
Step: 79400, Reward mean: 0.188, Reward std: 0.7434083669155197
Step: 79500, Reward mean: 0.202, Reward std: 0.7544507936240771
Step: 79600, Reward mean: 0.158, Reward std: 0.7436639025796532
Step: 79700, Reward mean: 0.22, Reward std: 0.7318469785412796
Step: 79800, Reward mean: 0.186, Reward std: 0.7611859168429221
Step: 79900, Reward mean: 0.214, Reward std: 0.7431042995434759
Step: 80000, Reward mean: 0.202, Reward std: 0.7356602476687183
Step: 80100, Reward mean: 0.234, Reward std: 0.7397594203523197
Step: 80200, Reward mean: 0.172, Reward std: 0.7364889680097048
Step: 80300, Reward mean: 0.174, Reward std: 0.7346591046192785
Step: 80400, Reward mean: 0.228, Reward std: 0.7238894943290722
Step: 80500, Reward mean: 0.22, Reward std: 0.7560423268574319
Step: 80600, Reward mean: 0.226, Reward std: 0.7313849875407615
Step: 80700, Reward mean: 0.138, Reward st